<a href="https://colab.research.google.com/github/cianc/AoC2023/blob/main/day11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [111]:
TEST = False #@param {type: "boolean"}
TEST_INPUT1 = 'day11_test1.txt'
TEST_INPUT2 = 'day11_test1.txt'
INPUT = 'day11_input1.txt'
PART=1 #@param {type: "integer"}


In [ ]:
import time

def get_input() -> list[str]:
  if PART == 1:
    input = TEST_INPUT1 if TEST else INPUT
  else:
    input = TEST_INPUT2 if TEST else INPUT

  with open(input, 'r') as f:
    rows = f.read().splitlines()
  return rows

def parse_map(input: list[str]) -> list[tuple[str, str]]:
  # Need to transpose rows and columns so that the first list index is
  # the x coordinate and the second is y like god intended
  map = [[] for i in range(len(input[0]))]
  galaxy_id = 1
  for row in input:
    for i, value in enumerate(row):
      if value == "#":
        value = str(galaxy_id)
        galaxy_id += 1
      map[i].append(value)

  x = 0
  while True:
    column = map[x]
    if all([cell=='.' for cell in column]):
      map[x] = ['c' for cell in column]
    x += 1
    if x >= len(map):
      break

  y = 0
  while True:
    row = [column[y] for column in map]
    if all([cell in ('.', 'c') for cell in row]):
      for column_index in range(len(map)):
        map[column_index][y] = 'r'
    y += 1
    if y >= len(map[0]):
      break

  return map

def print_map(map: list[tuple[str, str]]):
  for y in range(len(map[0])):
    row = [column[y] for column in map]
    print(''.join(row))

def calculate_distance_pairs(map: list[tuple[str, str]], ancient: bool=False) -> dict[tuple[str, str], int]:
  distances = {}

  galaxy_map = {}
  for x in range(len(map)):
    for y in range(len(map[0])):
      cell = map[x][y]
      if map[x][y].isnumeric():
        galaxy_map[cell] = (x,y)

  galaxy_names = galaxy_map.keys()
  for this_galaxy in galaxy_names:
    this_coords = galaxy_map[this_galaxy]
    for other_galaxy in [g for g in galaxy_names if g != this_galaxy]:
      other_coords = galaxy_map[other_galaxy]
      if distances.get(tuple(sorted((this_galaxy, other_galaxy))), None):
        continue

      horizontal_path = []
      horizontal_path_length = this_coords[0] - other_coords[0]
      if horizontal_path_length >= 0:
        max_x_point = this_coords
        min_x_point = other_coords
      else:
        max_x_point = other_coords
        min_x_point = this_coords
      for x_delta in range(abs(horizontal_path_length)):
        horizontal_path.append(map[min_x_point[0]+x_delta][min_x_point[1]])

      vertical_path = []
      vertical_path_length = this_coords[1] - other_coords[1]
      if vertical_path_length >= 0:
        max_y_point = this_coords
        min_y_point = other_coords
      else:
        max_y_point = other_coords
        min_y_point = this_coords
      for y_delta in range(abs(vertical_path_length)):
        vertical_path.append(map[min_y_point[0]][min_y_point[1]+y_delta])

      if ancient:
        expansion_addition = (1000000 - 1)
      else:
        expansion_addition = (2 - 1)
      expanded_column_count = len([cell for cell in horizontal_path if cell == 'c'])
      expanded_row_count = len([cell for cell in vertical_path if cell == 'r'])
      distance = (abs(this_coords[0] - other_coords[0]) +
                  abs(this_coords[1] - other_coords[1]) +
                  (expanded_column_count * expansion_addition) +
                  (expanded_row_count * expansion_addition))
      distances[tuple(sorted((this_galaxy, other_galaxy)))] = distance

  return distances

start_time = time.time()
input = get_input()
map = parse_map(input)
distance_pairs = calculate_distance_pairs(map, True)
print(f'sum of distances: {sum(distance_pairs.values())}')
end_time = time.time()
print(f'run time: {end_time - start_time} (s)')